In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read in all the words
words = open('../data/names.txt').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
# build vocabulary and mappings
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [113]:
# build dataset

def build_dataset(words):
    block_size = 3 # content length: how many characters to predict next one
    x, y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            x.append(context)
            y.append(ix)
            context = context[1:] + [ix] # crop and append

    x = torch.tensor(x)
    y = torch.tensor(y)
    print(x.shape, y.shape)
    return x, y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

xtr, ytr = build_dataset(words[:n1])
xdev, ydev = build_dataset(words[n1:n2])
xte, yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [43]:
x.shape, x.dtype, y.shape, y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [7]:
c = torch.rand((27, 2))

In [8]:
emb = c[x]
emb.shape

torch.Size([32, 3, 2])

In [9]:
w1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [10]:
h = torch.tanh(emb.view(-1, 6) @ w1 + b1)

In [11]:
h.shape

torch.Size([32, 100])

In [12]:
w2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [13]:
logits = h @ w2 + b2

In [14]:
counts = logits.exp()

In [15]:
prob = counts / counts.sum(1, keepdims=True)

In [16]:
prob[0].sum()

tensor(1.)

In [17]:
loss = -prob[torch.arange(32), y].log().mean()
loss

tensor(20.4580)

In [114]:
xtr.shape, ytr.shape

(torch.Size([182625, 3]), torch.Size([182625]))

In [131]:
g = torch.Generator().manual_seed(2147483647)
c = torch.randn((27, 10), generator=g)
w1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
w2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [c, w1, b1, w2, b2]

In [132]:
sum(p.nelement() for p in parameters) # total parameters

11897

In [133]:
for p in parameters:
    p.requires_grad = True

In [ ]:
stepi = []
lossi = []
stepi = []

In [138]:
for i in range(200000):
    # minibatch
    ix = torch.randint(0, xtr.shape[0], (32,))
    
    # forward pass
    emb = c[xtr[ix]]
    h = torch.tanh(emb.view(-1, 30) @ w1 + b1) # 32, 100
    logits = h @ w2 + b2 # 32, 27
    loss = F.cross_entropy(logits, ytr[ix])
    #print(loss.item())
    
    # backward
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
    
    # stats
    stepi.append(i)
    lossi.append(loss.log10().item())

print(loss.item())

2.3711721897125244


In [139]:
emb = c[xdev]
h = torch.tanh(emb.view(-1, 30) @ w1 + b1) # 32, 100
logits = h @ w2 + b2 # 32, 27
loss = F.cross_entropy(logits, ydev)
loss

tensor(2.1825, grad_fn=<NllLossBackward>)

In [141]:
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    out = []
    context = [0] * block_size
    while True:
        emb = c[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ w1 + b1)
        logits = h @ w2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))

carmah.
amille.
khirming.
taty.
skanden.
jazhutchemerynci.
geli.
nellara.
chaiir.
kaleig.
dham.
joce.
quinn.
sulin.
alianni.
wajero.
dearyni.
jaxee.
duraton.
dezi.
